In [1]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
from numpy.fft import fft, ifft
import numpy as np
import scipy
import seaborn as sns
import re
from tqdm.notebook import tqdm
import math

#IMPORTING FUNCTIONS FOR FEATURE EXTRACTION
import tsfresh
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import sklearn.linear_model as LinearModel

#IMPORTING FUNCTIONS FOR CLASSIFIERS
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, TimeSeriesSplit, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier

#IMPORTING FUNCTIONS FOR CLASSIFIERS METRICS
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

#OPTMIZE PARAMETHERS
import optuna

#IMPORTING FUNCTIONS FROM ./defs

from defs import defs_

In [2]:
#PATHS USED
path_quick_test = [
    "./database_raw/DatasetA/zzzAD1.txt",
    "./database_raw/DatasetA/zzzAD30.txt",
    "./database_raw/DatasetA/zzzAD12.txt",
    "./database_raw/DatasetA/zzzAU.txt",
]

path = glob.glob("./database_raw/DatasetA/zzzA*.txt", recursive=True)

pathB = glob.glob("./database_raw/DatasetB/zzzB*.txt", recursive=True)

# Qty of row to be aggrouped 
samples = 1024
# Qty of coef 
qty_coef = 30

# Initiate DF's

### You need to create the folder under df_csv/ with name of the string !!!

In [4]:
# Create full database with all data available on DATASETA
df_final = defs_.prepareRegressionDatabase(path, samples, qty_coef=qty_coef);
# Instead of creating database every time, it was exported as a csv file named 'dataframe_final' on root of project, imported using following code
df_final.to_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframe_regression_coef_'+str(qty_coef)+'_final');
#######
## Example of code to import database as pd
# df_final = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframe_final', index_col=0, dtype='unicode');
#######
### time coef20: 12m 21s
### time coef30: 14m 23s

df_final

,coeff_0_S1,coeff_1_S1,coeff_2_S1,coeff_3_S1,coeff_4_S1,coeff_5_S1,coeff_6_S1,coeff_7_S1,coeff_8_S1,coeff_9_S1,...,coeff_21_S30,coeff_22_S30,coeff_23_S30,coeff_24_S30,coeff_25_S30,coeff_26_S30,coeff_27_S30,coeff_28_S30,coeff_29_S30,Scenario
0,-0.177819,0.043999,-0.317334,-0.176758,-0.247790,-0.053505,-0.014552,0.265169,-0.092948,0.345784,...,-0.359003,-0.466606,0.199702,-0.536595,-0.152330,-0.645169,-0.240431,-0.304045,-0.111334,1
1,-0.069947,0.014569,-0.430368,-0.241814,-0.342427,-0.065819,0.010218,0.199684,-0.142716,0.359821,...,-0.238097,-0.299390,0.350837,-0.195308,0.023379,-0.396999,-0.205659,-0.265155,-0.096454,1
2,-0.043893,-0.004843,-0.385547,-0.296048,-0.402523,-0.017170,-0.046550,0.245820,-0.123614,0.460966,...,-0.195288,-0.533097,0.179553,-0.416387,-0.086161,-0.543830,-0.251633,-0.368864,-0.030988,1
3,-0.158048,0.006540,-0.466334,-0.303809,-0.349925,-0.044167,-0.003909,0.209332,-0.101166,0.413534,...,-0.316137,-0.393010,0.301667,-0.255874,-0.107325,-0.454604,-0.199115,-0.286673,-0.252909,1
4,-0.078782,0.101918,-0.437935,-0.287375,-0.399652,-0.130289,-0.126550,0.107221,-0.088206,0.401228,...,-0.266549,-0.373104,0.209822,-0.340268,-0.088525,-0.563888,-0.127661,-0.322133,-0.196231,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,-0.098258,-0.231781,-0.204274,-0.093877,-0.165207,-0.141420,0.195567,0.095536,0.074824,0.109399,...,-0.105828,-0.194271,0.360806,-0.346532,0.114440,-0.440529,-0.002694,-0.062458,-0.070008,0
7932,-0.150855,-0.268952,-0.310578,-0.219548,-0.266383,-0.143688,0.160058,0.062521,0.158177,0.194136,...,-0.131544,-0.144440,0.472784,-0.314056,0.124986,-0.314348,0.073575,-0.088311,-0.171353,0
7933,-0.072083,-0.295446,-0.405527,-0.269524,-0.251478,-0.118135,0.183689,0.084808,0.072563,0.164958,...,-0.093129,-0.209652,0.469708,-0.291869,0.132838,-0.384156,0.044976,-0.173390,-0.188000,0
7934,-0.126305,-0.274596,-0.384859,-0.167091,-0.201595,-0.191190,0.151111,0.087098,0.069777,0.030236,...,-0.160453,-0.225533,0.437749,-0.268343,0.092903,-0.363578,0.008406,-0.134736,-0.117858,0


In [15]:
# Create pratical database with three different scenarios available on DATASETA, use to test code before apply on full database in order to save time
df_quick_test = defs_.prepareRegressionDatabase(path_quick_test, samples, qty_coef=qty_coef);
# Instead of creating database every time, it was exported as a csv file named 'dataframesimples' on root of project, imported using following code
df_quick_test.to_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframe_regression_coef_'+str(qty_coef)+'_simples');
# Import database on csv file using following code
#######
## Example of code to import database as pd
# df_quick_test = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframe_simples', index_col=0, dtype='unicode');
#######
### time: 1m 33s

df_quick_test

,coeff_0_S1,coeff_1_S1,coeff_2_S1,coeff_3_S1,coeff_4_S1,coeff_5_S1,coeff_6_S1,coeff_7_S1,coeff_8_S1,coeff_9_S1,...,coeff_11_S30,coeff_12_S30,coeff_13_S30,coeff_14_S30,coeff_15_S30,coeff_16_S30,coeff_17_S30,coeff_18_S30,coeff_19_S30,Scenario
0,0.229661,0.029035,0.156659,-0.082888,0.139679,-0.063482,-0.184231,-0.275294,-0.154858,-0.098377,...,-0.504418,-0.514620,0.074956,-0.559486,-0.282882,-0.633803,-0.215768,-0.286246,-0.164153,1
1,0.226114,0.085449,0.161600,0.038953,0.004103,0.006793,-0.171089,-0.345459,-0.096588,-0.156187,...,-0.243763,-0.358807,0.299363,-0.278820,-0.080874,-0.446423,-0.161447,-0.238524,-0.032760,1
2,0.172765,0.006034,0.196995,0.087147,0.100559,0.047717,-0.285724,-0.545545,-0.340706,-0.316859,...,-0.272394,-0.673271,0.029841,-0.463385,-0.123232,-0.601990,-0.235433,-0.361600,0.004792,1
3,0.254877,-0.037947,0.220175,0.118779,0.233483,0.082073,-0.065925,-0.346545,-0.137024,-0.257596,...,-0.446192,-0.615268,0.081487,-0.434884,-0.279462,-0.559020,-0.159166,-0.287994,-0.194237,1
4,0.226537,0.028509,0.255326,0.059527,0.220564,0.150634,-0.089457,-0.374216,-0.160531,-0.123631,...,-0.380159,-0.516437,0.038324,-0.425970,-0.220754,-0.619256,-0.127345,-0.293884,-0.153089,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.050939,-0.120839,-0.166252,-0.324640,-0.071408,0.062067,-0.233672,-0.276335,-0.343122,0.208372,...,-0.014019,-0.347290,0.251712,-0.363252,0.116375,-0.489138,-0.000240,-0.072314,0.055474,0
1020,0.100610,0.060125,-0.005343,-0.233387,-0.049333,0.085645,-0.235135,-0.151654,-0.094969,0.391155,...,0.058880,-0.281459,0.398200,-0.377837,0.192160,-0.377328,0.079125,-0.095556,0.080026,0
1021,0.108114,-0.057814,-0.101018,-0.279807,0.008974,0.112333,-0.151791,-0.173488,-0.275010,0.243905,...,0.010517,-0.280031,0.416444,-0.378707,0.156120,-0.388744,0.111815,-0.165770,0.031694,0
1022,0.110551,0.070760,-0.049833,-0.222616,-0.006769,0.191327,-0.239453,-0.281070,-0.351102,0.147501,...,-0.045158,-0.329366,0.365537,-0.248272,0.138955,-0.405027,0.025756,-0.064861,-0.011693,0


In [18]:
# Create full database with all data available on DATASETA
df_finalB = defs_.prepareRegressionDatabase(pathB, samples, qty_coef=qty_coef);
# Instead of creating database every time, it was exported as a csv file named 'dataframe_final' on root of project, imported using following code
df_finalB.to_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframeB_regression_coef_' + str(qty_coef)  +'_final');
#######
## Example of code to import database as pd
# df_final = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframe_final', index_col=0, dtype='unicode');
#######
### time coef8: 10m 07s
### time coef16: 11m 29s

df_finalB

,coeff_0_S1,coeff_1_S1,coeff_2_S1,coeff_3_S1,coeff_4_S1,coeff_5_S1,coeff_6_S1,coeff_7_S1,coeff_8_S1,coeff_9_S1,...,coeff_11_S30,coeff_12_S30,coeff_13_S30,coeff_14_S30,coeff_15_S30,coeff_16_S30,coeff_17_S30,coeff_18_S30,coeff_19_S30,Scenario
0,0.154579,-0.173269,-0.082833,-0.106676,-0.007851,-0.053127,-0.119572,-0.296915,0.001543,-0.106854,...,-0.237169,-0.461389,0.204944,-0.423035,-0.210515,-0.555617,-0.186996,-0.326284,-0.162858,1
1,0.116162,-0.094464,0.074490,-0.070081,-0.062352,0.099952,-0.103726,-0.380804,-0.096472,-0.098292,...,-0.181254,-0.607569,0.180114,-0.517746,-0.110679,-0.570444,-0.188810,-0.426130,-0.050268,1
2,0.374840,0.050265,0.231095,-0.015144,-0.030359,-0.080244,-0.158980,-0.439141,-0.018062,-0.125757,...,-0.223551,-0.597080,0.340961,-0.210107,-0.068929,-0.563827,-0.039654,-0.197106,0.027930,1
3,0.171098,-0.203029,-0.003715,-0.204220,-0.046984,-0.096518,-0.159012,-0.390235,-0.099886,-0.151489,...,-0.363464,-0.550268,0.270541,-0.425782,-0.182741,-0.648053,-0.052674,-0.334880,-0.141654,1
4,0.101812,0.018681,0.088039,-0.018912,-0.004932,0.124179,-0.131056,-0.375432,-0.131764,-0.057662,...,-0.276703,-0.506563,0.254467,-0.279464,-0.141784,-0.598737,-0.028350,-0.224754,-0.102313,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,0.138915,-0.053902,-0.069768,-0.177611,0.012342,0.019068,-0.115731,-0.101958,-0.318776,0.165039,...,-0.233037,-0.401309,0.152059,-0.470444,-0.020875,-0.530980,-0.105663,-0.279901,-0.112626,0
7932,0.188134,-0.055236,-0.134415,-0.177941,-0.035664,0.074124,-0.171257,-0.080388,-0.138336,0.218568,...,-0.227776,-0.476371,0.121949,-0.562185,-0.106643,-0.474419,-0.183392,-0.345830,-0.169484,0
7933,0.057796,0.010016,0.053156,-0.210553,-0.003927,0.055902,-0.229441,-0.195240,-0.129589,0.368670,...,0.012706,-0.247761,0.336678,-0.332448,0.058687,-0.417531,-0.015472,-0.153585,-0.060324,0
7934,-0.000450,0.046743,0.098804,-0.249163,0.058326,0.173123,-0.206152,-0.259194,-0.091493,0.206887,...,-0.018693,-0.264698,0.315280,-0.345455,0.103836,-0.443012,0.117438,-0.073052,0.045000,0
